In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("test_pyspark") \
    .config("spark.driver.memory", "100g") \
    .config("spark.executor.memory", "100g") \
    .config("spark.sql.orc.enableVectorizedReader", "false") \
    .config("spark.sql.parquet.columnarReaderBatchSize", "256") \
    .config("spark.sql.orc.columnarReaderBatchSize", "256") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 14:06:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#spark.stop()

In [2]:
spark.table("iiif_manifests").count()

339927

In [3]:
spark.table("iiif_manifests").printSchema()

root
 |-- @context: string (nullable = true)
 |-- @id: string (nullable = true)
 |-- @type: string (nullable = true)
 |-- behavior: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- collections: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- @id: string (nullable = true)
 |    |    |-- @type: string (nullable = true)
 |    |    |-- label: string (nullable = true)
 |    |    |-- manifests: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- @id: string (nullable = true)
 |    |    |    |    |-- @type: string (nullable = true)
 |    |    |    |    |-- label: string (nullable = true)
 |    |    |    |    |-- thumbnail: struct (nullable = true)
 |    |    |    |    |    |-- @id: string (nullable = true)
 |    |    |    |    |    |-- @type: string (nullable = true)
 |    |    |    |    |    |-- service: struct (nullable = true)
 |    |    |    |    |    |    |-- @context

In [2]:
from pyspark.sql import functions as F
from typing import Tuple
import os

# extract id and manifest id from filename with a UDF

def extract_ids(filename: str) -> Tuple[str, str]:
    # Extract the last part of the filename (before .json)
    base_name = os.path.basename(filename)
    id_part, manifest_id = base_name.split('_', 1)
    manifest_id = manifest_id.rsplit('.', 1)[0]  # Remove .json
    return id_part, manifest_id

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, StructType, StructField
schema = StructType([
    StructField("id", StringType(), True),
    StructField("manifest_id", StringType(), True)
])

extract_ids_udf = udf(extract_ids, schema)

df = spark.table("iiif_manifests")

df = df.withColumn("ids", extract_ids_udf(F.col("filename")))

df2 = (df
       .withColumn("id", F.col("ids.id"))
       .withColumn("manifest_id", F.col("ids.manifest_id"))
       # 2) drop the original struct
       .drop("ids")
       )

# df2.sample(False, 0.001).limit(10).show(truncate=True)

Py4JJavaError: An error occurred while calling o35.table.
: java.util.concurrent.ExecutionException: java.net.ConnectException: Call From ip-172-31-22-145/172.31.22.145 to localhost:8020 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at org.sparkproject.guava.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at org.sparkproject.guava.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at org.sparkproject.guava.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at org.sparkproject.guava.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at org.sparkproject.guava.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2410)
	at org.sparkproject.guava.cache.LocalCache$Segment.loadSync(LocalCache.java:2380)
	at org.sparkproject.guava.cache.LocalCache$Segment.lockedGetOrLoad(LocalCache.java:2342)
	at org.sparkpro

In [3]:
df2.write.saveAsTable(
    "iiif_manifests_w_ids",
    mode="overwrite")

In [6]:
spark.sql("AlTER TABLE iiif_manifests RENAME TO iiif_manifests_old")
# Rename the table
spark.sql("AlTER TABLE iiif_manifests_w_ids RENAME TO iiif_manifests")


DataFrame[]

In [ ]:

# Drop the original table
spark.sql("DROP TABLE IF EXISTS iiif_manifests_old")

In [ ]:
from wc_simd.utility import spark_path

spark.table("iiif_manifests_w_ids").write.parquet(spark_path(
    "../data/iiif_manifests.parquet"), mode="overwrite")

In [4]:
spark.table("iiif_manifests_w_ids").count()

339927